In [29]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import os
import json

In [32]:
api_key = "843263a42ef422fedcc5a1abcbbfbaa7"

In [33]:
save_path = r"D:\workspace-Jwp\Final_Data"

In [105]:
def load_movies(year, end_page):
    Year = year
    PageN = range(1, end_page+1)
    
    movie_list = []
    for page in PageN:
        url = f"http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=430156241533f1d058c603178cc3ca0e&openStartDt={Year}&openEndDt={Year}&itemPerPage=100&curPage={page}"
        res = requests.get(url)
        movie_list.extend(res.json()["movieListResult"]["movieList"])
    
    df_movies = pd.DataFrame(movie_list)
    directors = pd.DataFrame([*df_movies.directors])
    
    temp = []
    for cols in directors:
        ls = []
        for i in directors[cols]:
            try:
                ls.append(i["peopleNm"])
            except:
                ls.append(" ")
        temp.append(ls)

    direc_values = list(zip(*temp))
    direc_values = list(map(lambda x: ", ".join(x).replace(",  ", ""), direc_values))
    df_movies.directors = direc_values
    
    return df_movies

In [106]:
df_movies = load_movies(2018, 19)

In [108]:
df_movies.head()

,movieCd,movieNm,movieNmEn,prdtYear,openDt,typeNm,prdtStatNm,nationAlt,genreAlt,repNationNm,repGenreNm,directors,companys
0,20174442,뷰티풀 데이즈,Beautiful Days,2018,20181121,장편,개봉,"한국,프랑스",드라마,한국,드라마,윤재호,"[{'companyCd': '20101149', 'companyNm': '(주)페퍼..."
1,20184678,배반의 장미,Too Hot to Die,2018,20181018,장편,개봉,한국,코미디,한국,코미디,박진영,"[{'companyCd': '20100947', 'companyNm': '(주)태원..."
2,20168766,종이 비행기,Paper Planes,2016,20180131,장편,개봉,한국,드라마,한국,드라마,"노홍식, 필동","[{'companyCd': '20112272', 'companyNm': '(주)파란..."
3,20182466,퀴어영화 뷰티풀,QUEER MOVIE Beautiful,2018,20181006,장편,개봉,한국,"멜로/로맨스,코미디",한국,멜로/로맨스,백인규,"[{'companyCd': '20140513', 'companyNm': '99fil..."
4,20177981,밤치기,Hit the Night,2017,20181101,장편,개봉,한국,멜로/로맨스,한국,멜로/로맨스,정가영,"[{'companyCd': '20175761', 'companyNm': '(주)레진..."


In [8]:
df_movies.companys

0       [{'companyCd': '20100947', 'companyNm': '(주)태원...
1       [{'companyCd': '20101149', 'companyNm': '(주)페퍼...
2       [{'companyCd': '20112272', 'companyNm': '(주)파란...
3       [{'companyCd': '20140513', 'companyNm': '99fil...
4       [{'companyCd': '20175761', 'companyNm': '(주)레진...
                              ...                        
1867    [{'companyCd': '20040209', 'companyNm': '영화사 온...
1868                                                   []
1869                                                   []
1870                                                   []
1871                                                   []
Name: companys, Length: 1872, dtype: object

In [9]:
companys = pd.DataFrame([*df_movies.companys])

In [109]:
companys.head()

,0,1,2
0,"{'companyCd': '20100947', 'companyNm': '(주)태원엔...",None,None
1,"{'companyCd': '20101149', 'companyNm': '(주)페퍼민...",None,None
2,"{'companyCd': '20112272', 'companyNm': '(주)파란프...",None,None
3,"{'companyCd': '20140513', 'companyNm': '99film'}",None,None
4,"{'companyCd': '20175761', 'companyNm': '(주)레진엔...",None,None


In [12]:
companys[companys[2].isnull() == False]

,0,1,2
172,"{'companyCd': '20100278', 'companyNm': '(주)제이케...","{'companyCd': '20188461', 'companyNm': '그리고픽처스'}","{'companyCd': '20110854', 'companyNm': '씨제이이앤엠..."
197,"{'companyCd': '20139999', 'companyNm': '(주)무비락'}","{'companyCd': '20140755', 'companyNm': '(주)도서관...","{'companyCd': '20173761', 'companyNm': '(주)푸른나..."
198,"{'companyCd': '20100123', 'companyNm': '(주)싸이더스'}","{'companyCd': '20124237', 'companyNm': '(주)엠씨엠씨'}","{'companyCd': '20062469', 'companyNm': '(주)더블앤..."
654,"{'companyCd': '20141515', 'companyNm': '(주)영화사...","{'companyCd': '20161781', 'companyNm': '(주)스튜디...","{'companyCd': '20140888', 'companyNm': '모티브픽쳐스..."
1261,"{'companyCd': '20157148', 'companyNm': '(주)케이프...","{'companyCd': '20101396', 'companyNm': '버티고필름'}","{'companyCd': '20123175', 'companyNm': '플로우식스(..."
1428,"{'companyCd': '20139800', 'companyNm': '영화사 이지'}","{'companyCd': '20186741', 'companyNm': '한국영상대학교'}","{'companyCd': '20173382', 'companyNm': '(주)컴퍼니..."


In [13]:
com_code = []
com_name = []
for cols in companys:
    ls1 = []
    ls2 = []
    for i in companys[cols]:
        try:
            ls1.append(i["companyCd"])
            ls2.append(i["companyNm"])
        except:
            ls1.append(" ")
            ls2.append(" ")
    com_code.append(ls1)
    com_name.append(ls2)

In [14]:
code = list(zip(*com_code))
name = list(zip(*com_name))

In [15]:
list(zip(*com_code, *com_name))[:5]

[('20100947', ' ', ' ', '(주)태원엔터테인먼트', ' ', ' '),
 ('20101149', ' ', ' ', '(주)페퍼민트앤컴퍼니', ' ', ' '),
 ('20112272', ' ', ' ', '(주)파란프로덕션', ' ', ' '),
 ('20140513', ' ', ' ', '99film', ' ', ' '),
 ('20175761', ' ', ' ', '(주)레진엔터테인먼트', ' ', ' ')]

In [16]:
name[:5]

[('(주)태원엔터테인먼트', ' ', ' '),
 ('(주)페퍼민트앤컴퍼니', ' ', ' '),
 ('(주)파란프로덕션', ' ', ' '),
 ('99film', ' ', ' '),
 ('(주)레진엔터테인먼트', ' ', ' ')]

In [17]:
df_movies["companyCd"] = code
df_movies["companyNm"] = name

In [18]:
list(zip(code, name))[:5]

[(('20100947', ' ', ' '), ('(주)태원엔터테인먼트', ' ', ' ')),
 (('20101149', ' ', ' '), ('(주)페퍼민트앤컴퍼니', ' ', ' ')),
 (('20112272', ' ', ' '), ('(주)파란프로덕션', ' ', ' ')),
 (('20140513', ' ', ' '), ('99film', ' ', ' ')),
 (('20175761', ' ', ' '), ('(주)레진엔터테인먼트', ' ', ' '))]

In [25]:
df_movies = df_movies.drop("companys", axis=1)

In [28]:
df_movies.to_csv(os.path.join(save_path, "movies_2018_3.csv"), encoding="utf-8", index=None)

In [ ]:
df_movies.to_csv(os.path.join(save_path, "movies_2018_3_euc.csv"), encoding="euc-kr", index=None)